In [ ]:
from minitrade.backtest import Strategy

class MyStrategy(Strategy):
    def init(self):
        self.roc = self.I(self.data.ta.roc(10), name='ROC')  # calculate the rate of change of the close price

    def next(self):
        self.alloc.assume_zero()  # initialize the allocation to zero
        roc = self.roc.df.iloc[-1]  # get the latest rate of change
        (self.alloc.bucket['equity']
            .append(roc.sort_values(ascending=False), roc > 0)  # add to bucket stocks with positive ROC ranked in descending order
            .keep_up_to(3)  # keep up to the top 3 stocks
            .weight_explicitly(1/3)  # allocate 1/3 of portfolio value to each asset in the bucket
            .apply())  # apply the allocation
        self.rebalance(cash_reserve=0.01)  # rebalance the portfolio

from minitrade.datasource import QuoteSource
from minitrade.backtest import Backtest

yahoo = QuoteSource.get_source('Yahoo')
data = yahoo.daily_data['MMM', 'AXP', 'AAPL', 'BA', 'CVX', start='2023-01-01']
bt = Backtest(data, MyStrategy, cash=10000)
bt.run()

bt.plot(plot_allocation=True)
